# Spark
[http://spark.apache.org/docs/2.1.0/api/python/pyspark.html](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html)

## Installation
- Install GOW
    - [https://github.com/bmatzelle/gow/releases](https://github.com/bmatzelle/gow/releases)
- Install Spark
    - [https://www.apache.org/dyn/closer.lua/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz](https://www.apache.org/dyn/closer.lua/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop3.2.tgz)
    - create a directory to hold the spark installation, and move the downloaded .tgz file there
    - navigate to that directory in the command prompt
    - upzip using `gzip -d filename.tgz`
    - untar using `tar xvf filename.tar`
    - navigate to the new spark folder in the cmd prompt
    - cd into bin and add winutils using command below
    - `curl -k -L -o winutils.exe https://github.com/steveloughran/winutils/tree/master/hadoop-3.0.0/bin/winutils.exe?raw=true`
    - ensure Java 8 is installed to path [https://medium.com/big-data-engineering/how-to-install-apache-spark-2-x-in-your-pc-e2047246ffc3](https://medium.com/big-data-engineering/how-to-install-apache-spark-2-x-in-your-pc-e2047246ffc3)
    - in command prompt, set spark variables
    - `setx SPARK_HOME C:\Opt\spark\spark-3.0.0-preview2-bin-hadoop3.2`
    - `setx HADOOP_HOME C:\Opt\spark\spark-3.0.0-preview2-bin-hadoop3.2`
    - `setx PYSPARK_DRIVER_PYTHON iptyhon` try jupyter here if issues
    - `setx PYSPARK_DRIVER_PYTHON_OPTS notebook`
    - add `C:\Opt\spark\spark-3.0.0-preview2-bin-hadoop3.2\bin\` to the PATH variables

## Spark Framework
- Platform for cluster computing
- Data is spread over multiple nodes in each cluster
    - each node only handles a small amount of data, increasing speed
    - results in parallel computing, as computations are carried out simultaneously
- Considerations
    - is my data too big for one machine?
    - can my computations be easily parallelized?

## Connecting to a Cluster
- The master computer is connected to all other nodes
    - this manages the data and splits up the calculations
- The other computers in the cluster are workers
    - workers get data and calculations to run from the master
    - they return the results to the master
- Create an instance of the `SparkContext` class

## Spark Data Structures
- The Resilient Distributed Dataset (RDD) is spark's core data structure
    - this lets spark split the data across multiple nodes in a cluster
    - RDD's are hard to work with directly
- Spark Dataframe abstraction
    - built on top of RDD
    - easier to use than direct RDD's

## Working with Spark Dataframes
- Create a `SparkSession` from the `SparkContext`
    - `SparkContext` is the connection to the cluster
    - `SparkSession` is the interface to that connection

## Creating a Spark Session
- Creating multiple sessions and contexts can cause issues
    - the `SparkSession.builder.getOrCreate()` method will return an existing session if there is one, and create one if there isn't one yet